In [1]:
from ipynb.fs.full.setup import *
from ipynb.fs.full.A_matrix import *
from ipynb.fs.full.boundaryCellDisplacement import *
from ipynb.fs.full.boundaryCellTraction import *

6
5
6
5
Shear Modulus (mu): 76923076923.07692
Lame Modulus (Lambda): 115384615384.61539


In [2]:
# Write a function that reads each corner edge BC (traction or displacement) and applies the correct a and b terms

#This will take in edges array, example: [t,l] -> this means top left corner
#It will then search BC_settings 

def cell_corner_BCs(A_matrix, b_matrix, k, edges, xy, U_previous, U_old, U_old_old):

    a_N = A(xy).a_N
    a_S = A(xy).a_S
    a_E = A(xy).a_E
    a_W = A(xy).a_W

    #Overrule a terms for on the boundary

    for edge in edges:

        if edge == "t":

            if BC_settings("t").traction:

                # Apply traction settings to North term
                a_N = 0        
            
            elif BC_settings("t").fixed_displacement:

                # Apply fixed boundary setings to North term
                a_N = boundaryCellDisplacement(["t"], xy).a_N        

        if edge == "b":

            if BC_settings("b").traction:

                # Apply traction settings to South term
                a_S = 0

            elif BC_settings("b").fixed_displacement:

                # Apply fixed boundary setings to South term
                a_S = boundaryCellDisplacement(["b"], xy).a_S

        if edge == "r":

            if BC_settings("r").traction:

                # Apply traction settings to East term
                a_E = 0

            elif BC_settings("r").fixed_displacement:

                # Apply fixed boundary setings to East term
                a_E = boundaryCellDisplacement(["r"], xy).a_E

        if edge == "l":

            if BC_settings("l").traction:

                # Apply traction settings to West term
                a_W = 0

            elif BC_settings("l").fixed_displacement:

                # Apply fixed boundary setings to West term
                a_W = boundaryCellDisplacement([edge], xy).a_W

    if transient:
        a_P = (rho*dx*dy/(dt**2)) + a_N + a_S + a_E + a_W

    else:
        a_P = a_N + a_S + a_E + a_W

    #Aply a terms to matrices

    A_matrix[k,k] = a_P

    for edge in edges:

        if edge == "t":
        
            A_matrix[k,index(k).s] = - a_S

        elif edge == "b":

            A_matrix[k,index(k).n] = - a_N

        elif edge == "l":

            A_matrix[k,index(k).e] = - a_E

        elif edge == "r":
            
            A_matrix[k,index(k).w] = - a_W 

    def cornercorner(edges, corner_placement, uv, U_previous, k):

        # This is where the extrapolation occurs
 
        if uv == "u":
            uv_i = 0
            xy = "x"
        elif uv == "v":
            uv_i = 1
            xy = "y"

        disp = displacement(k, U_previous, uv_i)

        # Top left:
        if edges == ["t", "l"]: 
            if corner_placement == "NW":
                corner = ((3/2)*disp.P - (1/2)*disp.SE)

            elif corner_placement == "SE":
                corner = A.corner(corner_placement, uv, U_previous, k)
            
            else:
                for i in [["NE", 0], ["SW", 1]]:            
                    if corner_placement == i[0]:
                        if BC_settings(edges[i[1]]).traction:
                            corner = boundaryCellTraction.corner([edges[i[1]]], corner_placement, uv, U_previous, k)
                        elif BC_settings(edges[i[1]]).fixed_displacement:
                            corner = boundaryCellDisplacement.corner([edges[i[1]]], corner_placement, uv, U_previous, k)


        # Top Right:
        if edges == ["t", "r"]: 
            if corner_placement == "NE":
                corner = ((3/2)*disp.P - (1/2)*disp.SW)

            elif corner_placement == "SW":
                corner = A.corner(corner_placement, uv, U_previous, k)
            
            else:
                for i in [["NW", 0], ["SE", 1]]:
                    if corner_placement == i[0]:
                        if BC_settings(edges[i[1]]).traction:
                            corner = boundaryCellTraction.corner([edges[i[1]]], corner_placement, uv, U_previous, k)
                        elif BC_settings(edges[i[1]]).fixed_displacement:
                            corner = boundaryCellDisplacement.corner([edges[i[1]]], corner_placement, uv, U_previous, k)


        # Bottom Left:
        if edges == ["b", "l"]:
            if corner_placement == "SW":
                corner = ((3/2)*disp.P - (1/2)*disp.NE)

            elif corner_placement == "NE":
                corner = A.corner(corner_placement, uv, U_previous, k)
            
            else: 
                for i in [["SE", 0], ["NW", 1]]:   
                    if corner_placement == i[0]:
                        if BC_settings(edges[i[1]]).traction:
                            corner = boundaryCellTraction.corner([edges[i[1]]], corner_placement, uv, U_previous, k)
                        elif BC_settings(edges[i[1]]).fixed_displacement:
                            corner = boundaryCellDisplacement.corner([edges[i[1]]], corner_placement, uv, U_previous, k)

        # Bottom Right:
        if edges == ["b", "r"]: 
            if corner_placement == "SE":
                corner = ((3/2)*disp.P - (1/2)*disp.NW)

            elif corner_placement == "NW":
                corner = A.corner(corner_placement, uv, U_previous, k)
            
            else:
                for i in [["SW", 0], ["NE", 1]]:            
                    if corner_placement == i[0]:
                        if BC_settings(edges[i[1]]).traction:
                            corner = boundaryCellTraction.corner([edges[i[1]]], corner_placement, uv, U_previous, k)
                        elif BC_settings(edges[i[1]]).fixed_displacement:
                            corner = boundaryCellDisplacement.corner([edges[i[1]]], corner_placement, uv, U_previous, k)



        return corner   

    # b diff

    def b_diff(edges, k, xy, U_previous):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"
            
        def N_term():

            if (edges[0] != "t") | ((edges[0] == "t") & (BC_settings("t").fixed_displacement)):
                N_term = Sfy*A.coef(xy, "N", uv)*(
                    (cornercorner(edges, "NE", uv, U_previous, k) - cornercorner(edges, "NW", uv, U_previous, k))
                    /dx)

            elif BC_settings("t").traction:   
                if (xy == "x") : N_term =  Sfy*tr_top_x  
                if (xy == "y") : N_term =  Sfy*tr_top_y            

            return N_term

        def S_term():

            if (edges[0] != "b") | ((edges[0] == "b") & (BC_settings("b").fixed_displacement)):
                S_term = Sfy*A.coef(xy, "S", uv)*(
                    (cornercorner(edges, "SE", uv, U_previous, k) - cornercorner(edges, "SW", uv, U_previous, k))
                    /dx)

            elif BC_settings("b").traction: 
                if (xy == "x") : S_term =  Sfy*tr_bottom_x 
                if (xy == "y") : S_term =  Sfy*tr_bottom_y

            return S_term

        def E_term():

            if (edges[1] != "r") | ((edges[1] == "r") & (BC_settings("r").fixed_displacement)):
                E_term = Sfx*A.coef(xy, "E", uv)*(
                            (cornercorner(edges, "NE", uv, U_previous, k) - cornercorner(edges, "SE", uv, U_previous, k))
                            /dy)
                
            elif BC_settings("r").traction: 
                if (xy == "x") : E_term =  Sfx*tr_right_x  
                if (xy == "y") : E_term =  Sfx*tr_right_y

            return E_term

        def W_term():

            if (edges[1] != "l") | ((edges[1] == "l") & (BC_settings("l").fixed_displacement)):
                W_term = Sfx*A.coef(xy, "W", uv)*(
                (cornercorner(edges, "NW", uv, U_previous, k) - cornercorner(edges, "SW", uv, U_previous, k))
                /dy)

            elif BC_settings("l").traction: 
                if (xy == "x") : W_term =  Sfx*tr_left_x  
                if (xy == "y") : W_term =  Sfx*tr_left_y

            return W_term

        b_diffusion = N_term() + S_term() + E_term() + W_term()

        return b_diffusion




    b_matrix[k] =(
            A.b_temp(U_old, U_old_old, k, xy)
            +
            b_diff(edges, k, xy, U_previous)
        )  

    return A_matrix, b_matrix    

